In [ ]:
import pandas as pd
import sklearn
import yfinance as yf

In [ ]:
TICKERS = ['GC=F', 'DX-Y.NYB', '^GSPC', '^TNX', '^VIX']
START_DATE = '2001-01-01'
TARGET_COL = 'Gold_Price'

# --- 1. Data Retrieval and Preprocessing ---
def get_and_clean_data(tickers, start_date):
    """Downloads data, selects 'Close' prices, and cleans up the DataFrame."""
    print("Downloading data...")
    all_data = yf.download(tickers, start=start_date, interval='1d')

    # Isolate 'Close' prices and clean up
    df = all_data['Close'].dropna(axis=1, how='all')
    df = df.ffill()
    df = df.rename(columns={'GC=F': TARGET_COL})

    # Drop any raw ticker columns that failed to load
    for ticker in tickers:
        if ticker in df.columns and ticker != TARGET_COL:
            if df[ticker].isnull().all():
                 df = df.drop(columns=[ticker])

    return df

In [ ]:
df_raw = get_and_clean_data(TICKERS, START_DATE)

C:\Users\Martijn\AppData\Local\Temp\ipykernel_17096\1320821376.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  all_data = yf.download(tickers, start=start_date, interval='1d')
[*********************100%***********************]  5 of 5 completed


In [ ]:
import sys
from pathlib import Path

# Add the parent directory of 'features' to sys.path
training_dir = Path.cwd().parent
if str(training_dir) not in sys.path:
	sys.path.insert(0, str(training_dir))

from features.build_features import build_features

In [ ]:
df = build_features(df_raw)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import mlflow
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neural_network import MLPRegressor

In [ ]:
numeric_features = [
    'Gold_Price_L0',
    'Gold_Price_L5',
    'Gold_Price_L20',
    'Gold_Price_10D_MA',
    'Gold_Price_50D_MA',
    'Gold_Log_Return',
    'Gold_Log_Return_20D_Vol',
    'DXY_Level',
    '10Y_Yield_Level',
    'VIX_Level',
    'SPX_Log_Return'
]

categorical_features = [
    'day',
    'month',
    'year',
    'dayofweek',
]


In [ ]:
preprocessor_tree = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
preprocessor_linear = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
    ]
)

In [ ]:
candidates = {
    "rf": Pipeline([("prep", preprocessor_tree),
                    ("model", RandomForestRegressor())]),
    "xgb": Pipeline([("prep", preprocessor_tree),
                     ("model", XGBRegressor(eval_metric="rmse"))]),
    "nn": Pipeline([("prep", preprocessor_linear),
                    ("model", MLPRegressor(max_iter=1000))])
}

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import mlflow

def train_all_models(df, n_splits=5):

    print("🔧 Building features...")
    df_features = build_features(df)

    X = df_features.drop("Target_Gold_Price", axis=1)
    y = df_features["Target_Gold_Price"]

    print(f"📊 Total rows after feature engineering: {len(df_features)}")

    tscv = TimeSeriesSplit(n_splits=n_splits)

    results = {}

    print(f"⏳ Starting training for {len(candidates)} models with {n_splits} time-series splits...\n")

    for name, pipeline in candidates.items():
        print(f"====================")
        print(f"🚀 Training model: {name}")
        print(f"====================")

        scores = []

        for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
            print(f"\n--- Fold {fold+1}/{n_splits} ---")
            print(f"Train size: {len(train_idx)}, Test size: {len(test_idx)}")

            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            with mlflow.start_run(run_name=f"{name}_fold{fold}"):

                print("🧠 Fitting model...")
                pipeline.fit(X_train, y_train)

                print("📈 Predicting...")
                preds = pipeline.predict(X_test)

                rmse = mean_squared_error(y_test, preds)
                print(f"📉 RMSE (fold {fold+1}): {rmse:.4f}")

                mlflow.log_metric("RMSE", rmse)
                scores.append(rmse)

                rsquared = pipeline.score(X_test, y_test)
                mlflow.log_metric("R2", rsquared)

        avg_score = sum(scores) / len(scores)
        results[name] = avg_score

        print(f"\n✅ Finished model: {name}")
        print(f"🏆 Average RMSE: {avg_score:.4f}\n")

    print("🎉 All models trained successfully!")
    print("📊 Final Results:", results)

    return results


In [ ]:
result = train_all_models(df_raw)

🔧 Building features...
📊 Total rows after feature engineering: 6253
⏳ Starting training for 3 models with 5 time-series splits...

🚀 Training model: rf

--- Fold 1/5 ---
Train size: 1043, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 1): 103535.0804

--- Fold 2/5 ---
Train size: 2085, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 2): 1186988.6241

--- Fold 3/5 ---
Train size: 3127, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 3): 700845.7542

--- Fold 4/5 ---
Train size: 4169, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 4): 604344.5015

--- Fold 5/5 ---
Train size: 5211, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 5): 1610669.7977

✅ Finished model: rf
🏆 Average RMSE: 841276.7516

🚀 Training model: xgb

--- Fold 1/5 ---
Train size: 1043, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 1): 102004.2166

--- Fold 2/5 ---
Train size: 2085, Test size: 1042
🧠 Fitting model...
📈 Pred

c:\Programming\Ironhack\projects\gold-price-mlops\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


📈 Predicting...
📉 RMSE (fold 1): 11467.9500

--- Fold 2/5 ---
Train size: 2085, Test size: 1042
🧠 Fitting model...


c:\Programming\Ironhack\projects\gold-price-mlops\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


📈 Predicting...
📉 RMSE (fold 2): 9684.5610

--- Fold 3/5 ---
Train size: 3127, Test size: 1042
🧠 Fitting model...


c:\Programming\Ironhack\projects\gold-price-mlops\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


📈 Predicting...
📉 RMSE (fold 3): 325.6013

--- Fold 4/5 ---
Train size: 4169, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 4): 307.9784

--- Fold 5/5 ---
Train size: 5211, Test size: 1042
🧠 Fitting model...
📈 Predicting...
📉 RMSE (fold 5): 908.4217

✅ Finished model: nn
🏆 Average RMSE: 4538.9025

🎉 All models trained successfully!
📊 Final Results: {'rf': 841276.7515957551, 'xgb': 733755.7024791832, 'nn': 4538.902482630095}
